
# Análise de Apólices — Notebook
Este notebook organiza as análises solicitadas em etapas funcionais e reproduzíveis, sem interatividade (apenas Pandas/NumPy/Matplotlib).


In [2]:

# %% [params]
from pathlib import Path
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Caminhos de entrada/saída
INPUT_CSV = Path('apolices.csv')   # ajuste se necessário
OUT_DIR = Path('analise_apolices_nb')
OUT_DIR.mkdir(exist_ok=True, parents=True)

pd.options.display.float_format = lambda x: f'{x:,.2f}'


In [5]:
df = read_apolices(INPUT_CSV)
df

,Unnamed: 0,Proposta,Apólice,Endosso,Endosso Associado,Descrição do Endosso,Status da Apólice,ID do Grupo do Ramo,Número do Ramo,Nome do Grupo do Ramo,...,Data de Nascimento,Sexo,Tipo de Pessoa,Código do Produto,Nome do Produto,CEP,Cidade,UF,PA Venda,Motivo do Cancelamento
0,0,205297,2023011905310165265,000002T,0.00,ENDOSSO DE CANCELAMENTO SEM RESTITUIÇÃO DE PREMIO,CANCELADO,5,31,AUTOMÓVEL - CASCO,...,1992-05-15,MASCULINO,PF,99,AUTOMOVEL/CAMINHAO,87075-630,MARINGÁ,PR,4340_65,FALTA DE PAGAMENTO
1,1,206282,2023011905310166450,000001T,0.00,ENDOSSO DE CANCELAMENTO SEM RESTITUIÇÃO DE PREMIO,CANCELADO,5,31,AUTOMÓVEL - CASCO,...,1965-08-11,FEMININO,PF,99,AUTOMOVEL/CAMINHAO,87113-310,SARANDI,PR,434004,FALTA DE PAGAMENTO
2,2,206548,2023011905310166395,000001T,0.00,ENDOSSO DE CANCELAMENTO COM RESTITUIÇÃO DE PREMIO,CANCELADO,5,31,AUTOMÓVEL - CASCO,...,1957-07-21,FEMININO,PF,99,AUTOMOVEL/CAMINHAO,87600-000,NOVA ESPERANÇA,PR,4340_68,SOLICITAÇÃO DO SEGURADO/CORRETOR
3,3,205481,2023011905310165615,000001R,0.00,ENDOSSO DE RESTITUIÇÃO DE PREMIO,CANCELADO,5,31,AUTOMÓVEL - CASCO,...,1983-07-20,FEMININO,PF,99,AUTOMOVEL/CAMINHAO,87240-000,TERRA BOA,PR,434032,SOLICITAÇÃO DO SEGURADO/CORRETOR
4,4,205481,2023011905310165615,000001T,0.00,ENDOSSO DE CANCELAMENTO COM RESTITUIÇÃO DE PREMIO,CANCELADO,5,31,AUTOMÓVEL - CASCO,...,1983-07-20,FEMININO,PF,99,AUTOMOVEL/CAMINHAO,87240-000,TERRA BOA,PR,434032,SOLICITAÇÃO DO SEGURADO/CORRETOR
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
174902,174902,987,1011401000188,2000001,NaN,CANCELAMENTO,VIGENTE,1,14,PATRIMONIAL,...,1973-02-18,FEMININO,PF,17,RESIDENCIAL FÁCIL,19430000,MARABÁ PAULISTA,SP,NaN,SOLICITAÇÃO DA SEGURADORA
174903,174903,99,1011401000006,0,NaN,APÓLICE,VIGENTE,1,14,PATRIMONIAL,...,1966-05-04,MASCULINO,PF,17,RESIDENCIAL FÁCIL,86067970,LONDRINA,PR,NaN,NaN
174904,174904,990,1011401000232,2000000,NaN,CANCELAMENTO,VIGENTE,1,14,PATRIMONIAL,...,1995-12-14,FEMININO,PF,17,RESIDENCIAL FÁCIL,13456625,SANTA BÁRBARA D'OESTE,SP,NaN,SOLICITAÇÃO DA SEGURADORA
174905,174905,992,1011401000236,2000000,NaN,CANCELAMENTO,VIGENTE,1,14,PATRIMONIAL,...,1997-05-23,MASCULINO,PF,17,RESIDENCIAL FÁCIL,09321255,MAUÁ,SP,NaN,SOLICITAÇÃO DA SEGURADORA


In [4]:

# %% [utils]
from typing import Optional, Tuple, Dict, Any

def read_apolices(path: Path) -> pd.DataFrame:
    df = pd.read_csv(path, low_memory=False)
    df.columns = [c.strip() for c in df.columns]
    return df

def parse_date_cols(df: pd.DataFrame) -> pd.DataFrame:
    # Identificar colunas com "Data" no nome e criar colunas parseadas
    for c in [c for c in df.columns if 'data' in c.lower()]:
        df[c + ' (parsed)'] = pd.to_datetime(df[c], errors='coerce')
    return df

def detect_columns(df: pd.DataFrame) -> Dict[str, Optional[str]]:
    cols = df.columns.tolist()
    def pick(fn):
        try:
            return next((c for c in cols if fn(c)), None)
        except StopIteration:
            return None
    return {
        'apolice': pick(lambda c: c.lower() in ['apólice','apolice','nr_apolice','numero da apólice','número da apólice']),
        'endosso': pick(lambda c: c.lower() in ['endosso','nr_endosso','numero do endosso','número do endosso']),
        'desc_endosso': pick(lambda c: 'descri' in c.lower() and 'endosso' in c.lower()),
        'status': pick(lambda c: ('status' in c.lower()) and ('apólice' in c.lower() or 'apolice' in c.lower())),
        'motivo_cancel': pick(lambda c: ('motivo' in c.lower()) and ('cancel' in c.lower())),
        'produto_nome': pick(lambda c: 'nome do produto' in c.lower()),
        'produto_cod': pick(lambda c: 'código do produto' in c.lower() or 'codigo do produto' in c.lower() or 'cd_produto' in c.lower()),
        'ramo_grupo': pick(lambda c: 'grupo do ramo' in c.lower()),
        'uf': pick(lambda c: c.strip().upper() == 'UF'),
        'cidade': pick(lambda c: 'cidade' in c.lower()),
        'cep': pick(lambda c: 'cep' in c.lower()),
        'sexo': pick(lambda c: 'sexo' in c.lower()),
        'tipo_pessoa': pick(lambda c: 'tipo de pessoa' in c.lower() or 'tipo_pessoa' in c.lower()),
        'dt_emissao': pick(lambda c: 'emiss' in c.lower() and ' (parsed)' not in c.lower()),
        'dt_inicio': pick(lambda c: ('início' in c.lower() or 'inicio' in c.lower()) and ' (parsed)' not in c.lower()),
        'dt_fim': pick(lambda c: 'fim' in c.lower() and ' (parsed)' not in c.lower()),
        'dt_nasc': pick(lambda c: 'nascimento' in c.lower() or 'dt_nascimento' in c.lower())
    }

def to_numeric_br(series: pd.Series) -> pd.Series:
    s = series.astype(str).str.replace('.', '', regex=False).str.replace(',', '.', regex=False)
    return pd.to_numeric(s, errors='coerce')

def add_numeric_candidates(df: pd.DataFrame) -> pd.DataFrame:
    candidates = [c for c in df.columns if any(k in c.lower() for k in ['valor','vl_','prêmio','premio','iof','custo','fracionamento',' receita',' is','is '])]
    for c in candidates:
        df[c + ' (num)'] = to_numeric_br(df[c])
    return df

def contains_ci(s: pd.Series, pat: str) -> pd.Series:
    return s.astype(str).str.lower().str.contains(pat, na=False)

def monthly_count(s: pd.Series) -> pd.DataFrame:
    d = pd.to_datetime(s, errors='coerce')
    return d.dt.to_period('M').value_counts().sort_index().to_frame('qtd')

def save_table(obj: Any, name: str) -> Path:
    path = OUT_DIR / f'{name}.csv'
    if obj is None:
        return path
    if isinstance(obj, pd.Series):
        obj.to_frame().to_csv(path, sep=';')
    else:
        obj.to_csv(path, index=True)
    return path

def plot_simple(series_or_df, title: str):
    # opcional — não interativo
    plt.figure()
    if isinstance(series_or_df, pd.Series):
        series_or_df.plot(kind='barh')
    else:
        series_or_df.plot(kind='barh')
    plt.title(title)
    plt.tight_layout()
    plt.show()


In [30]:

# %% [load + prepare]
df = read_apolices(INPUT_CSV)
df = parse_date_cols(df)
df = add_numeric_candidates(df)
cols = detect_columns(df)

summary_base = {
    'qtd_registros': int(len(df)),
    'qtd_colunas': int(len(df.columns)),
    'colunas_principais': cols
}
summary_base


{'qtd_registros': 174907,
 'qtd_colunas': 41,
 'colunas_principais': {'apolice': 'Apólice',
  'endosso': 'Endosso',
  'desc_endosso': 'Descrição do Endosso',
  'status': 'Status da Apólice',
  'motivo_cancel': 'Motivo do Cancelamento',
  'produto_nome': 'Nome do Produto',
  'produto_cod': 'Código do Produto',
  'ramo_grupo': 'ID do Grupo do Ramo',
  'uf': 'UF',
  'cidade': 'Cidade',
  'cep': 'CEP',
  'sexo': 'Sexo',
  'tipo_pessoa': 'Tipo de Pessoa',
  'dt_emissao': 'Data de Emissão',
  'dt_inicio': 'Data de Início da Vigência',
  'dt_fim': 'Data de Fim da Vigência',
  'dt_nasc': 'Data de Nascimento'}}

In [31]:

# %% [quality]
dup_rate = None
if cols['apolice'] and cols['endosso']:
    dup_rate = float(df.duplicated(subset=[cols['apolice'], cols['endosso']], keep=False).mean())

# Missing
missing_pct = df.isna().mean().sort_values(ascending=False)

# Datas
issues = {}
if cols['dt_inicio'] and cols['dt_fim']:
    di = pd.to_datetime(df[cols['dt_inicio']], errors='coerce')
    dfim = pd.to_datetime(df[cols['dt_fim']], errors='coerce')
    issues['vig_fim_antes_inicio'] = int((dfim < di).sum())

if cols['dt_emissao'] and cols['dt_inicio']:
    de = pd.to_datetime(df[cols['dt_emissao']], errors='coerce')
    di = pd.to_datetime(df[cols['dt_inicio']], errors='coerce')
    issues['emissao_depois_inicio'] = int((de > di).sum())

save_table(missing_pct, '01_missing_pct')

{
    'duplicidade_apolice_endosso_rate': dup_rate,
    'inconsistencias_datas': issues
}


{'duplicidade_apolice_endosso_rate': 0.0004116473325824581,
 'inconsistencias_datas': {'vig_fim_antes_inicio': 76,
  'emissao_depois_inicio': 66214}}

In [32]:

# %% [temporal]
temporal = {}
if cols['dt_emissao']:
    temporal['emissoes_mensal'] = monthly_count(pd.to_datetime(df[cols['dt_emissao']], errors='coerce'))
    save_table(temporal['emissoes_mensal'], '02_emissoes_mensal')
if cols['dt_inicio']:
    temporal['inicio_vigencia_mensal'] = monthly_count(pd.to_datetime(df[cols['dt_inicio']], errors='coerce'))
    save_table(temporal['inicio_vigencia_mensal'], '03_inicio_vigencia_mensal')
if cols['dt_fim']:
    temporal['fim_vigencia_mensal'] = monthly_count(pd.to_datetime(df[cols['dt_fim']], errors='coerce'))
    save_table(temporal['fim_vigencia_mensal'], '04_fim_vigencia_mensal')

temporal


{'emissoes_mensal':                    qtd
 Data de Emissão       
 2024-06          12405
 2024-07          12568
 2024-08          11875
 2024-09          12399
 2024-10          10728
 2024-11           9002
 2024-12          13655
 2025-01          12787
 2025-02          11891
 2025-03          12283
 2025-04          13210
 2025-05          12387
 2025-06          14159
 2025-07          14440
 2025-08           1118,
 'inicio_vigencia_mensal':                             qtd
 Data de Início da Vigência     
 2002-06                       1
 2014-09                       1
 2015-09                       1
 2016-09                       1
 2017-09                       1
 ...                         ...
 2025-11                       8
 2026-02                       1
 2026-04                       2
 2026-07                       2
 2027-05                       3
 
 [68 rows x 1 columns],
 'fim_vigencia_mensal':                           qtd
 Data de Fim da Vigência      
 2002-

In [33]:

# %% [cancelamentos]
cancel_mask = (
    (contains_ci(df[cols['status']], 'cancel') if cols['status'] else False) |
    (contains_ci(df[cols['desc_endosso']], 'cancel') if cols['desc_endosso'] else False) |
    (contains_ci(df[cols['motivo_cancel']], 'cancel') if cols['motivo_cancel'] else False)
)

df['__is_cancelado'] = cancel_mask

cancelamento = {'taxa_global': float(cancel_mask.mean())}

# Top motivos
if cols['motivo_cancel']:
    top_motivos = (
        df.loc[cancel_mask, cols['motivo_cancel']]
          .astype(str).str.strip().str.upper()
          .value_counts().head(15)
    )
    cancelamento['top_motivos'] = top_motivos
    save_table(top_motivos, '05_top_motivos_cancel')

# Tempo até cancelamento (aproximação)
tempo = {}
if cols['dt_inicio']:
    di = pd.to_datetime(df[cols['dt_inicio']], errors='coerce')
    if cols['dt_fim']:
        dt_cancel = pd.to_datetime(df[cols['dt_fim']], errors='coerce')
    elif cols['dt_emissao']:
        dt_cancel = pd.to_datetime(df[cols['dt_emissao']], errors='coerce')
    else:
        dt_cancel = pd.Series([pd.NaT]*len(df))
    delta = (dt_cancel - di).dt.days
    tempo = {
        'dias_medio': float(delta.loc[cancel_mask].mean(skipna=True)) if cancel_mask.any() else None,
        'dias_mediana': float(delta.loc[cancel_mask].median(skipna=True)) if cancel_mask.any() else None,
        'q1_q3': (
            float(delta.loc[cancel_mask].quantile(0.25)),
            float(delta.loc[cancel_mask].quantile(0.75))
        ) if cancel_mask.any() else None,
        'pct_cancel_em_ate_30d': float((delta.loc[cancel_mask] <= 30).mean()) if cancel_mask.any() else None
    }
    cancelamento['tempo_ate_cancelamento'] = tempo

def cancel_rate_table(group_col: Optional[str], min_volume=100) -> Optional[pd.DataFrame]:
    if not group_col or group_col not in df.columns:
        return None
    g = (
        df.assign(x_cancel=df['__is_cancelado'].astype(int))
          .groupby(group_col, dropna=False)
          .agg(qtd=('__is_cancelado','size'), cancelados=('x_cancel','sum'))
    )
    g['taxa_cancel'] = g['cancelados'] / g['qtd']
    g = g[g['qtd'] >= min_volume].sort_values('taxa_cancel', ascending=False)
    return g

tbl_prod = cancel_rate_table(cols['produto_nome'], min_volume=100)
tbl_ramo = cancel_rate_table(cols['ramo_grupo'], min_volume=100)
tbl_uf = cancel_rate_table(cols['uf'], min_volume=200)

save_table(tbl_prod, '06_cancel_por_produto')
save_table(tbl_ramo, '07_cancel_por_ramo_grupo')
save_table(tbl_uf, '08_cancel_por_uf')

cancelamento


{'taxa_global': 0.26209356972562564,
 'top_motivos': Motivo do Cancelamento
 FALTA DE PAGAMENTO                  17561
 NAN                                 14831
 SOLICITAÇÃO DO SEGURADO             12354
 SOLICITAÇÃO DO SEGURADO/CORRETOR      624
 ERRO DE EMISSÃO                       138
 SINISTRO                              101
 SOLICITAÇÃO DA SEGURADORA              71
 INDENIZAÇÃO INTEGRAL                   63
 REEMISSÃO A PEDIDO DO SEGURADO         63
 DECISÃO DA SEGURADORA                  11
 ALTERAÇÃO DE SUBVENÇÃO                 11
 ERRO DE SISTEMA                         9
 MOTIVOS TÉCNICOS                        3
 ALTERAÇÃO DE RISCO                      2
 Name: count, dtype: int64,
 'tempo_ate_cancelamento': {'dias_medio': 348.8325552986344,
  'dias_mediana': 365.0,
  'q1_q3': (365.0, 365.0),
  'pct_cancel_em_ate_30d': 0.014157322978927621}}

In [34]:

# %% [distribuicao]
dist = {}
def top_count(col, n=20):
    if not col:
        return None
    return df[col].astype(str).value_counts().head(n)

dist['top_produtos'] = top_count(cols['produto_nome'])
dist['top_ramo_grupo'] = top_count(cols['ramo_grupo'])
dist['top_uf'] = top_count(cols['uf'])

# salvar
save_table(dist['top_produtos'], '09_top_produtos')
save_table(dist['top_ramo_grupo'], '10_top_ramo_grupo')
save_table(dist['top_uf'], '11_top_uf')

dist


{'top_produtos': Nome do Produto
 RESIDENCIAL SEGURO COMPLETO             16337
 SANCOR VIDA COOPERADO EXCLUSIVO         16209
 SANCOR SEGUROS EMPRESARIAL              15446
 SANCOR SEGUROS VIDA MULHER              14246
 SANCOR SEGUROS VIDA GLOBAL              14166
 RESIDENCIAL SEGURO ESSENCIAL             8984
 RESIDENCIAL SEGURO FÁCIL II              7968
 SANCOR VIDA COOPERADO EXCLUSIVO V1       7681
 AUTOMOVEL/CAMINHAO                       7671
 SANCOR VIDA COOPERADO ESPECIAL           7651
 RESIDENCIAL SANCOR DIGITAL  II           5978
 SANCOR SEGUROS VIDA FÁCIL (PLANO II)     5553
 SANCOR SEGUROS VIDA (COLETIVO)           5442
 SANCOR SEGUROS AP FÁCIL (INDIVIDUAL)     5211
 SANCOR VIDA COOPERADO ESPECIAL V1        3297
 VIDA INDIVIDUAL PERSONALIZADO            3275
 SANCOR SEGUROS AP PREMIADO - BITPAGG     2786
 SANCOR SEGUROS VIDA IND. CORRENTISTA     2648
 AP PREMIADO SANCOR DIGITAL               2316
 SANCOR VIDA COOPERADO ESSENCIAL          2182
 Name: count, dtype: int64,

In [35]:

# %% [financeiro]
num_cols = [c for c in df.columns if c.endswith(' (num)')]
col_premio = next((c for c in num_cols if ('prêmio' in c.lower() or 'premio' in c.lower())), None)
col_custo = next((c for c in num_cols if 'custo' in c.lower()), None)

financeiro = {}
if col_premio:
    financeiro['premio_total'] = float(df[col_premio].sum(skipna=True))
    if cols['produto_nome']:
        top_premio_prod = df.groupby(cols['produto_nome'])[col_premio].sum(min_count=1).sort_values(ascending=False).head(20)
        financeiro['premio_por_produto_top'] = top_premio_prod
        save_table(top_premio_prod, '12_premio_por_produto_top')
    if cols['ramo_grupo']:
        top_premio_ramo = df.groupby(cols['ramo_grupo'])[col_premio].sum(min_count=1).sort_values(ascending=False).head(20)
        financeiro['premio_por_ramo_top'] = top_premio_ramo
        save_table(top_premio_ramo, '13_premio_por_ramo_top')
    if cols['uf']:
        top_premio_uf = df.groupby(cols['uf'])[col_premio].sum(min_count=1).sort_values(ascending=False).head(20)
        financeiro['premio_por_uf_top'] = top_premio_uf
        save_table(top_premio_uf, '14_premio_por_uf_top')

if col_premio and col_custo:
    financeiro['margem_bruta_aprox'] = float((df[col_premio] - df[col_custo]).sum(skipna=True))

financeiro


{'premio_total': 27708904906.0,
 'premio_por_produto_top': Nome do Produto
 SANCOR SEGUROS PRESTAMISTA TM VINCULADO     8506889723
 SANCOR SEGUROS EMPRESARIAL                  1644255451
 SANCOR SEGUROS PRESTAMISTA TM VARIAVEL      1472002084
 SANCOR SEGUROS VIDA GLOBAL                  1456949338
 SANCOR SEGUROS VIDA RURAL TM VINCULADO      1403194503
 AUTOMOVEL/CAMINHAO                          1243902563
 RESIDENCIAL SEGURO COMPLETO                 1138907349
 SANCOR VIDA COOPERADO EXCLUSIVO             1017984943
 SANCOR SEGUROS VIDA RURAL TM VARIAVEL        890829056
 MULTIRRISCO PLUS CREDCOOP                    876055518
 MULTIRRISCO PLUS VERÃO SANCOR UNICOOB 11     763389897
 SANCOR SEGUROS VIDA MULHER                   608959830
 SANCOR VIDA COOPERADO EXCLUSIVO V1           606814546
 SANCOR VIDA COOPERADO ESPECIAL               527637392
 MULTIRRISCO PLUS INVERNO CREDCOOP 11         516228003
 RESIDENCIAL SEGURO ESSENCIAL                 471237327
 SANCOR EMPRESARIAL LMI ÚNICO

In [36]:

# %% [perfil]
perfil = {}
if cols['sexo']:
    perfil['sexo'] = df[cols['sexo']].astype(str).str.upper().value_counts(dropna=False)
    save_table(perfil['sexo'], '15_perfil_sexo')

if cols['tipo_pessoa']:
    perfil['tipo_pessoa'] = df[cols['tipo_pessoa']].astype(str).str.upper().value_counts(dropna=False)
    save_table(perfil['tipo_pessoa'], '16_perfil_tipo_pessoa')

if cols['dt_nasc']:
    dt_nasc = pd.to_datetime(df[cols['dt_nasc']], errors='coerce')
    idade = np.floor((pd.Timestamp.today() - dt_nasc).dt.days / 365.25)
    bins = [0, 18, 25, 35, 45, 55, 65, 75, 200]
    labels = ['<=18','19-25','26-35','36-45','46-55','56-65','66-75','>75']
    faixa = pd.cut(idade, bins=bins, labels=labels, right=True, include_lowest=True)
    perfil['faixa_etaria'] = faixa.value_counts().sort_index()
    save_table(perfil['faixa_etaria'], '17_perfil_faixa_etaria')

perfil


{'sexo': Sexo
 MASCULINO    78823
 FEMININO     56376
 NAN          39708
 Name: count, dtype: int64,
 'tipo_pessoa': Tipo de Pessoa
 PF    135180
 PJ     39727
 Name: count, dtype: int64,
 'faixa_etaria': Data de Nascimento
 <=18       694
 19-25     7533
 26-35    27091
 36-45    37244
 46-55    30130
 56-65    22262
 66-75     8166
 >75       2826
 Name: count, dtype: int64}

In [37]:

# %% [riscos]
def top_cancel(table: pd.DataFrame, n=10) -> pd.DataFrame:
    if table is None or table.empty:
        return table
    return table.sort_values('taxa_cancel', ascending=False).head(n)

riscos = {
    'produtos_maior_cancel': top_cancel(pd.read_csv(OUT_DIR/'06_cancel_por_produto.csv', index_col=0), 10) if (OUT_DIR/'06_cancel_por_produto.csv').exists() else None,
    'uf_maior_cancel': top_cancel(pd.read_csv(OUT_DIR/'08_cancel_por_uf.csv', index_col=0), 10) if (OUT_DIR/'08_cancel_por_uf.csv').exists() else None
}

# salvar
if riscos['produtos_maior_cancel'] is not None:
    save_table(riscos['produtos_maior_cancel'], '18_produtos_maior_cancel')
if riscos['uf_maior_cancel'] is not None:
    save_table(riscos['uf_maior_cancel'], '19_uf_maior_cancel')

riscos


{'produtos_maior_cancel':                                        qtd  cancelados  taxa_cancel
 Nome do Produto                                                    
 RESIDENCIAL SEGURO EXCLUSIVO           701         308         0.44
 RESIDENCIAL SEGURO FÁCIL II           7968        3399         0.43
 RESIDENCIAL SEGURO ESSENCIAL          8984        3515         0.39
 RESIDENCIAL SEGURO COMPLETO          16337        6382         0.39
 SANCOR SEGUROS EMPRESARIAL FÁCIL       779         281         0.36
 EMPRESARIAL SANCOR DIGITAL            1003         360         0.36
 RESIDENCIAL SANCOR DIGITAL  II        5978        2075         0.35
 VIDA INDIVIDUAL FÁCIL PLUS            1046         359         0.34
 SANCOR SEGUROS EMPRESARIAL           15446        5216         0.34
 SANCOR SEGUROS VIDA FÁCIL (PLANO I)   1733         537         0.31,
 'uf_maior_cancel':        qtd  cancelados  taxa_cancel
 UF                                 
 AP    1396         706         0.51
 SP   13577     

In [38]:

# %% [summary]
resumo = {
    'registros': int(len(df)),
    'colunas': int(len(df.columns)),
    'taxa_cancel_global': float(df['__is_cancelado'].mean()) if '__is_cancelado' in df.columns else None,
    'artefatos_dir': str(OUT_DIR)
}
resumo


{'registros': 174907,
 'colunas': 42,
 'taxa_cancel_global': 0.26209356972562564,
 'artefatos_dir': 'analise_apolices_nb'}